<a href="https://colab.research.google.com/github/dagpadraget/rakdnngit/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
cd /content

/content


In [32]:
!rm -fr rakdnngit
!rm -fr tmp
!git clone https://github.com/dagpadraget/rakdnngit

Cloning into 'rakdnngit'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 116 (delta 57), reused 85 (delta 29), pack-reused 0
Receiving objects: 100% (116/116), 13.27 MiB | 7.30 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [33]:
cd rakdnngit

/content/rakdnngit


In [34]:
!pip3 install -r requirement.txt

In [0]:
mkdir tmp

In [38]:
cd tmp

/content/rakdnngit/tmp


In [0]:
mkdir rakdnngit

In [41]:
cd /content/rakdnngit

/content/rakdnngit


In [0]:
!python3 main.py

Using TensorFlow backend.
number of sequences= 27335 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-01-06 13:48:30.577182: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-01-06 13:48:30.612145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-01-06 13:48:30.612837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-01-06 13:48:30.613165: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-01-06 13:48:30.615036: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-01-06 1

In [0]:
pwd

In [0]:
more localdata/hbshort.txt

In [0]:
ls -rla /content/tmp/rakdnngit/jobdir/rak_dnngit

In [0]:
# *********************************************************************************************************************
# Library imports
import numpy as np
import os
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Conv2D,LeakyReLU, Flatten, Dense,Reshape,Activation,Conv2DTranspose,Embedding,Dropout,LSTM
from keras.models import Model
from keras.models import load_model
from keras.optimizers import RMSprop
from keras.utils import np_utils
import re
import argparse
# *********************************************************************************************************************


In [0]:
ext_seq_length=5
storyfile="hblang.txt"

In [0]:
# *********************************************************************************************************************
# Load data from file
def load_data(datapath):
    filename=datapath+"/"+storyfile
    print("story: "+filename)
    with open(filename,encoding="utf-8-sig") as f:
        text =f.read()

    seq_length=ext_seq_length # sequence around sentence
    step=1 # prediction step forward

    start_story='| '*seq_length

    text=text.lower()
    text=start_story+text
    text=text.replace('\n\n\n\n\n',start_story)
    text=text.replace('\n',' ')
    #text=re.sub(' +','. ',text).strip()
    text=text.replace('..','.')
    text=re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_{|}~])',r' \1 ',text)
    text=re.sub('\s{2,}',' ',text)

    tokenizer=Tokenizer(char_level=False,filters='')
    tokenizer.fit_on_texts([text])
    total_words=len(tokenizer.word_index)+1
    token_list=tokenizer.texts_to_sequences([text])[0]

    return token_list, seq_length,step,total_words,start_story,tokenizer
# *********************************************************************************************************************

In [0]:
def generate_text2(seed_text,next_words,model,max_sequence_len,temp,start_story,tokenizer):
    token_list = np.array(tokenizer.texts_to_sequences([seed_text])[0])
    token_list=token_list[-ext_seq_length:]
    probs = model.predict(token_list.reshape(1,len(token_list)), verbose=0)[0]
    y_class = sample_with_temp(probs, temperature=temp)
    output_word = tokenizer.index_word[y_class] if y_class > 0 else ''
    return seed_text+"=>"+output_word

In [0]:
def generate_text22(seed_text,next_words,model,max_sequence_len,temp,start_story,tokenizer):
    token_list = np.array(tokenizer.texts_to_sequences([seed_text])[0])
    token_list=token_list[-ext_seq_length:]
    probs = model.predict(token_list.reshape(1,len(token_list)), verbose=0)[0]
    
    print(1)
    p0=-np.sort(-probs)
    print(2)
    i0=np.argsort(-probs)
    print(3)
    print("words probability",p0[0:9])
    print("words index=",i0[0:9])
    print(4)
    pos=0
    for k in i0:
      print("words in order=",k," : ",tokenizer.index_word[k],'p=',p0[pos])
      if pos>10:
        break
      pos+=1
    print(5)
    
    y_class = sample_with_temp(probs, temperature=temp)
    output_word = tokenizer.index_word[y_class] if y_class > 0 else ''
    return output_word


In [0]:
# *********************************************************************************************************************
def sample_with_temp(preds,temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probs=np.random.multinomial(1,preds,1)
    return np.argmax(probs)
# *********************************************************************************************************************


In [0]:
tmpdirname='/content/tmp/rakdnngit'

In [0]:
xbatch_size=32
xlearning_rate=0.01
xnum_epochs=1


In [0]:
class Argar:
    pcsim = True
    pcsimdir = tmpdirname
    batch_size=xbatch_size
    learning_rate=xlearning_rate
    num_epochs=xnum_epochs
    job_dir=tmpdirname+"/jobdir"

In [0]:
args=Argar()

In [0]:
fileandpath = os.path.join(args.job_dir, 'rak_dnngit')+"/allincluded.model"

try:
  model=load_model(fileandpath)
  #model.summary()      
  token_list, seq_length, step, total_words,start_story,tokenizer = load_data("/content/rakdnngit/localdata")
except:
  print("could not find "+fileandpath)
  
print("output from loaded model : *************************")
print(generate_text2("till handelsman och ha", 1, model, 10, 0.01, start_story,tokenizer))
print(generate_text2("gråsalva åt grisen och", 1, model, 10, 0.01, start_story, tokenizer))
print(generate_text2("hade de hamnat på", 1, model, 10, 0.01, start_story, tokenizer))
print(generate_text2("därför skrek han att", 1, model, 10, 0.01, start_story, tokenizer))

In [0]:
print(generate_text22("sett en råseglare förr och därför skrek han att de", 1, model, 10, 0.01, start_story,tokenizer))
print(generate_text22("sett en hamster och därför så skrek han att de", 1, model, 10, 0.01, start_story,tokenizer))